## 3. Data Preparation
📒 `2.0-rc-data-preparation.ipynb`

**Objetivo:** Transformar os dados brutos em um formato para uso em análise exploratória e modelagem.

⚙️ **Atividades:**
- Tratamento de valores ausentes;
- Substituição de valores inconsistentes
- Conversão de tipos
- Remoção de colunas irrelevantes ou redundantes
- Trata dados faltantes, padroniza formatos, remove duplicatas
- Tratamento de outliers extremos
- Combinação de múltiplas fontes
- Salvamento do dataset limpo (`data/processed/`)

In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline

# Setup para mostrar todas as colunas do dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Desabilita mensagens de FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [309]:
# Carrega Dataset renomeando as colunas

cols = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]


file_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(file_path, names=cols)

# Visualiza as 5 primeiras linhas
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [310]:
# Trata inconsistência dos Dados - Substitui "?" por Valores Nulos
df = df.replace("?", np.nan)

# Verificar os valores Nulos
df.isnull().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

In [311]:
df.dtypes

symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horsepower            object
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                 object
dtype: object

In [312]:
# Converte colunas númericas que estão como objetos
numeric_cols = ['normalized-losses','price','bore', 'stroke', 'horsepower', 'peak-rpm']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Converte Colunas para Inteiro
int_numeric_cols = ['normalized-losses', 'price', 'horsepower', 'peak-rpm']
for col in int_numeric_cols:
    df[col] = df[col].astype('Int64')

#Verifica os Tipos de Dados Após Correção
df.dtypes


symboling              int64
normalized-losses      Int64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower             Int64
peak-rpm               Int64
city-mpg               int64
highway-mpg            int64
price                  Int64
dtype: object

In [313]:
# Substitui Valores Ausentes para:

# Variáveis Numéricas
# Preencher com mediana ( menos sensível a outliers)
for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)


# Variáveis Categóricas
# Preencher com a moda
categ_cols = df.select_dtypes(include='object').columns.to_list()

for col_c in categ_cols:
    df[col_c].fillna(df[col_c].mode()[0], inplace=True)
    print(f"MODA: {col_c} = {df[col_c].mode()[0]}")


# Verifica os valores Nulos Novamente 
print("Valores Nulos após tratamento:")
df.isnull().sum()


MODA: make = toyota
MODA: fuel-type = gas
MODA: aspiration = std
MODA: num-of-doors = four
MODA: body-style = sedan
MODA: drive-wheels = fwd
MODA: engine-location = front
MODA: engine-type = ohc
MODA: num-of-cylinders = four
MODA: fuel-system = mpfi
Valores Nulos após tratamento:


symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64

In [314]:
# Aplica o Mapping na variável `symboling`
symboling_map = {
    -3: 'low',
    -2:'low',
    -1:'moderate',
    0: 'moderate',
    1:'moderate',
    2:'high',
    3:'high'
}

# Cria nova coluna com a classificação de risco agrupadas 
df['risk_classification'] = df['symboling'].map(symboling_map)

# Verifica os valores únicos
df['risk_classification'].unique()

array(['high', 'moderate', 'low'], dtype=object)

In [315]:
# Tratar Outliers na variável target (price)
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filtra valores excluindo os outliers (dentro dos 3 desvios padrões)
df = df[~((df['price'] < lower_bound) | (df['price'] > upper_bound))]

# Reseta Index
df.reset_index(drop=True, inplace=True)

# Mostra valores após manipulação
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   symboling            191 non-null    int64  
 1   normalized-losses    191 non-null    Int64  
 2   make                 191 non-null    object 
 3   fuel-type            191 non-null    object 
 4   aspiration           191 non-null    object 
 5   num-of-doors         191 non-null    object 
 6   body-style           191 non-null    object 
 7   drive-wheels         191 non-null    object 
 8   engine-location      191 non-null    object 
 9   wheel-base           191 non-null    float64
 10  length               191 non-null    float64
 11  width                191 non-null    float64
 12  height               191 non-null    float64
 13  curb-weight          191 non-null    int64  
 14  engine-type          191 non-null    object 
 15  num-of-cylinders     191 non-null    obj

In [316]:
# Classificação do Preço (Binning Price)

# Definir o número de bins. Lembrando Bins = ponto de corte - 1, neste exemplo, precisamos de 3bins correspondentes as faixas de preço
bins_price = np.linspace(df['price'].min(), df['price'].max(), 4)
group_prices = ['low', 'medium', 'high']

# Classifica preço em intervalos definidos acima
df['price-binned'] = pd.cut(df['price'], bins=bins_price, labels=group_prices, include_lowest=True ).astype('object')


In [319]:
# Consumo Médio  (avg-fuel-consumption) em Km/l
# Fator de Conversão Km/l =  1mpg * 0.425144
df['avg-fuel-consumption'] = (((df['city-mpg'] + df['highway-mpg']) * 0.425144) / 2).round(1)


In [320]:
# Salva o dataset tratado
path_to_save = "../data/processed/car_price_cleaned.csv"
df.to_csv(path_to_save, index=False)